In [134]:
from typing import List, Tuple
from datetime import datetime
import pandas as pd
import numpy as np
class GTM:
    """Класс ГТМов.
    """
    def __init__(self, useful_period:int, capex_value:float, premium_rate:float,name:str) -> None:
        """Конструктор

        Args:
            useful_period (int): Срок полезного использования, лет. Количество лет, которые начисляется амортизация
            capex_value (float): Стоимтсть кап. влажений, млн. Справочное значение для типа ГТМа
            premium_rate (float): Амортизационная премия, д. ед. Доля единовременного списания стоимсти 
                основных фондов первый год расчета амортизации. По сути просто цифра на которую надо умножить стоимость при
                при расчете налоговой амортизации. Как и срок полезного использования справочное значение для типа ГТМа
            name (str): Название ГТМа
        """        
        self.useful_period = useful_period
        self.capex_value = capex_value
        self.premium_rate = premium_rate
        self.name = name

    
class Capex:
    """Класс вектора капитальных вложений
        Данный класс позволяет получить вектора капитальных вложений ТОЛЬКО для ГТМов.
    """
    tax_rate:float = 0.022
    def __init__(self, date_begin:str, date_end:str, *GTM_arr:GTM) -> None:
        """Конструктор

        Args:
            date_begin (str): Дата начала ремонта ('%d.%m.%Y')
            date_end (str): Дата окончания ремонта ('%d.%m.%Y')
            GTM_arr (GTM): Список ГТМов.
        """
        self.date_begin = datetime.strptime(date_begin, '%d.%m.%Y')
        self.date_end = date_end
        self._list_gtm = GTM_arr

    @property
    def max_useful_period(self) -> int:
        """Максимальное количество лет полезного использования среди всех ГТМов. Геттер
        """
        return max(self._list_gtm, key=lambda x: x.useful_period).useful_period

    def _deprication(self,has_premium:bool )-> np.ndarray:
        """Амортизационные затраты, млн. р
            Механика расчета определяеться СТО 1084 и налоговым кодексом
        Args:
            has_premium (bool): наличие или отсуствие премии
        Returns:
            np.array: Список затрат по годам, млн для каждого ГТМа из self._list_gtm
                Размерность количество ГТМов x максимальный срок исользования + 1
        """
        res = np.zeros((len(self._list_gtm), self.max_useful_period+1))
        for ind_g, gtm in enumerate(self._list_gtm):
            premium = has_premium* gtm.premium_rate
            count_month = 12 - self.date_begin.month
            res[ind_g,0] = gtm.capex_value*(1-premium)/gtm.useful_period/(12/count_month) + gtm.capex_value*premium
            for ind_year in range(1,gtm.useful_period):
                res[ind_g,ind_year ] = gtm.capex_value*(1-premium)/gtm.useful_period
            #если максимальный срок 10 лет, нужно 11 временных точек, чтобы учесть как бы остаток
            res[ind_g, gtm.useful_period] =gtm.capex_value*(1-premium)/gtm.useful_period/(12/self.date_begin.month) 
        return res
    @property
    def acount_deprication(self)-> np.ndarray:
        """Амортизационные отлисления по бухгалтерскому учету, млн. р Сеттер

        Returns:
            np.array: Список затрат по годам, млн для каждого ГТМа из self._list_gtm
                Двухмерный массив размерностью ГТМов x максимальный срок исользования + 1
        """
        return self._deprication(has_premium=False)
    @property
    def summ_acount_deprication(self)-> np.ndarray:
        """Сумма бух. отчислений по всем ГТМам. Сеттер

        Returns:
            np.ndarray: Одномерный массив размерностью максимальный срок исользования + 1
        """
        return self.acount_deprication.T.sum(axis=1)
    
    @property
    def summ_tax_deprication(self)-> np.ndarray:
        """Сумма налоговых. отчислений по всем ГТМам. Сеттер

        Returns:
            np.ndarray: Одномерный массив размерностью максимальный срок исользования + 1
        """
        return self.tax_deprication.T.sum(axis=1)
    @property
    def tax_deprication(self)-> np.ndarray: return self._deprication(has_premium=True)
    @property
    def _base_fonds_cost(self)-> np.ndarray:
        """Стоимость базовых фондов по годам для всех ГТМов в сумме на начало и на конец года

        Returns:
            np.array: Размерность максимальный срок исользования + 1 Х 2
                по второй оси два значение - начало года и конец года
                это нужно для того чтобы потом полсчитать среднюю в году стоимость с учетом амортизации
        """
        res = np.zeros((self.max_useful_period+1,2))
        res[0,0] = 0
        res[0,1] = sum([g.capex_value for g in self._list_gtm]) - self.summ_acount_deprication[0]
        # return res
        for idx in range(1, self.max_useful_period+1):
            res[idx,0] = res[idx-1][1]
            res[idx,1] = res[idx][0] - self.summ_acount_deprication[idx]
        return res
    @property
    def _average_year_fonds_cost(self)-> np.ndarray: 
        """Стоимость базовых фондов по годам для всех ГТМов в сумме в среднем в году

        Returns:
            List[float]: _description_
        """
        return self._base_fonds_cost.T[0]/2 + self._base_fonds_cost.T[1]/2
    @property
    def tax_property(self)->np.ndarray: 
        """Объем налоговых выплат.
            Налоговая база - основные фонды (как 13% из зарплаты)
            tax_rate = 2.2% налоговая ставка для всех скваин всей России. Цифра из налогового кодекса
        Returns:
            List[float]: _description_
        """
        return self._average_year_fonds_cost * self.tax_rate
    @property
    def _get_df(self)->pd.DataFrame:
        """DataFrame для визуализации

        Returns:
            pd.DataFrame: DataFrame
        """
        res = {}
        for ind_g, g in enumerate(self._list_gtm):
            res[("Амортизация, млн. р", g.name,"бухг.")] = self.acount_deprication[ind_g]
        for ind_g, g in enumerate(self._list_gtm):
            res[("Амортизация, млн. р",g.name,"налог")] = self.tax_deprication[ind_g]
        res[("Амортизация, млн. р","сумма ","БУ(бух. учет)")] = self.summ_acount_deprication
        res[("Амортизация, млн. р","сумма","НУ(налог. учет).")] = self.summ_tax_deprication
        res[("Стоимость ОФ(осн. фонды), млн. р","сумма","начало года")] = self._base_fonds_cost.T[0]
        res[("Стоимость ОФ(осн. фонды), млн. р","сумма","конец года")] = self._base_fonds_cost.T[1]
        res[("Стоимость ОФ(осн. фонды), млн. р","ср. за год"," ")] = self._average_year_fonds_cost
        res[("Налог на имущество, млн. р"," "," ")] = self.tax_property
        df = pd.DataFrame(res, index=[self.date_begin.year + ind for ind in range(self.max_useful_period+1)] )
        pd.options.display.float_format = '{:.2f}'.format
        return df
    def __repr__(self) -> str:
        """аналог .ToString в питоне

        Returns:
            str: возврощяет понятное дело строку
        """
        return repr(self._get_df)


gtm_klk = GTM(3, 8, .1, "КЛК") 
# КЛК. Концентрическая лифтовая колонна. В скваэину спускают еще одну трубу чтобы скважина лучше фунциклтировала
gtm_uk = GTM(10, 30, .3, "УК")
#УК КЛК. Упровляющий комплекс КЛК. Тоже самое, но с IOT-мозгами. Поэтому и стоит 10 млн. По сути он сам умеет влючать и вылючать трубу

# gtm_tab = GTM(15, 99, .99, "Табурет") Табурет
c = Capex('01.01.2023', '01.02.2023', gtm_klk, gtm_uk)
print(c) # вывод в comd-promt в табличном понятном виде для экономиста
c._get_df.T.to_clipboard() # в буфер обмена в понятном  виде для экономиста

     Амортизация, млн. р                                                   \
                     КЛК    УК   КЛК    УК        сумма             сумма   
                   бухг. бухг. налог налог БУ(бух. учет) НУ(налог. учет).   
2023                2.44  2.75  3.00 10.93          5.19            13.93   
2024                2.67  3.00  2.40  2.10          5.67             4.50   
2025                2.67  3.00  2.40  2.10          5.67             4.50   
2026                0.22  3.00  0.20  2.10          3.22             2.30   
2027                0.00  3.00  0.00  2.10          3.00             2.10   
2028                0.00  3.00  0.00  2.10          3.00             2.10   
2029                0.00  3.00  0.00  2.10          3.00             2.10   
2030                0.00  3.00  0.00  2.10          3.00             2.10   
2031                0.00  3.00  0.00  2.10          3.00             2.10   
2032                0.00  3.00  0.00  2.10          3.00             2.10   

Механика капитальных затрат